In [1]:
# import gmt gravfft (calculate gravity effect of bathymetry-seamount interface)
# import simpeg 
# import choclo (Fatiando a Terra, 2024) (calculate components of gravitational tensor for the synthetic model)
# import grdflexure GMT module (wessel, 2019) (calculate flexural deformation of lithosphere)

import numpy as np
from scipy.interpolate import LinearNDInterpolator
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

from discretize import TreeMesh
from discretize.utils import mkvc, refine_tree_xyz, active_from_xyz
from simpeg.utils import plot2Ddata, model_builder
from simpeg import maps
from simpeg.potential_fields import gravity

## Outline
- download bathymetry data from GEBCO
- make model of seamount in the shape of Kolumbo volcano
- introduce magma chamber in location, size and density corresponding to results from previous seismic tomography studies
- calculate gravity gradient effect of bathymetry (use different data set for bathymetry than satellite altimetry, to ensure independence) using Parker's expansion for a range of reference crustal densities
- filter 
- subtract bathymetry gravity effect from satellite altimetry gravity gradient
- model flexural compensation of seamount ()


find out
-read seismo papers on Kolumbo volcano



### Import bathymetry data from GEBCO 2024 
(https://download.gebco.net)

In [13]:
#columns = ['x', 'y', 'z']
topo_file = pd.read_csv('./GEBCO_bathymetry/kolumbo_bathymetry.xyz', sep='\s', header=None)
topo_file.columns = ['x', 'y', 'z']

x_topo = topo_file['x']
y_topo = topo_file['y']
z_topo = topo_file['z']

xyz_topo = np.c_[x_topo, y_topo, z_topo]

/var/folders/kv/9c2wczl17lzbpv6zgms9ms480000gn/T/ipykernel_33242/1999046034.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  topo_file = pd.read_csv('./GEBCO_bathymetry/kolumbo_bathymetry.xyz', sep='\s', header=None)


### Define the Survey